<a href="https://colab.research.google.com/github/mbudisic/pstuts-rag/blob/main/evaluate_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from getpass import getpass

from dotenv import load_dotenv



In [2]:
!git clone https://github.com/mbudisic/pstuts-rag.git
%cd pstuts-rag
!source .venv/bin/activate
!uv sync
!pip install pstuts_rag

fatal: destination path 'pstuts-rag' already exists and is not an empty directory.
/content/pstuts-rag
Resolved 252 packages in 1ms
Audited 238 packages in 0.51ms


In [3]:
import pstuts_rag

In [4]:
%load_ext autoreload
%autoreload 2


In [5]:
from dataclasses import dataclass
@dataclass
class ApplicationParameters:
    filename = "data/test.json"
    embedding_model = "text-embedding-3-small"
    n_context_docs = 2

params = ApplicationParameters()

In [6]:
import os
load_dotenv()
from google.colab import userdata


def set_api_key_if_not_present(key_name, prompt_message=""):
    if len(prompt_message) == 0:
        prompt_message=key_name
    if key_name not in os.environ or not os.environ[key_name]:
        os.environ[key_name] = userdata.get(key_name)

set_api_key_if_not_present("OPENAI_API_KEY")
set_api_key_if_not_present("HF_USER")
set_api_key_if_not_present("HF_TOKEN")

# Data Preparation

First, we will read in the transcripts of the videos and convert them to Documents
with appropriate metadata.

In [7]:
from ast import Dict
import json

from pstuts_rag.loader import load_json_files
filename = ["../data/test.json","../data/dev.json"]
from typing import List, Dict, Any
data:List[Dict[str,Any]] = await load_json_files(filename)


In [8]:
[ d["title"] for d in data ]

[]

## R - retrieval

Let's hit it with a semantic chunker.

In [9]:
from pstuts_rag.datastore import DatastoreManager
from qdrant_client import QdrantClient

client = QdrantClient(":memory:")
from sentence_transformers import SentenceTransformer
embeddings = SentenceTransformer("Snowflake/snowflake-arctic-embed-l")
retriever_factory = DatastoreManager(qdrant_client=client,name="local_test", embeddings=embeddings)
if retriever_factory.count_docs() == 0:
    await retriever_factory.populate_database(raw_docs=data)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/85.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

AttributeError: 'SentenceTransformer' object has no attribute 'embed_query'

## A - Augmentation

We need to populate a prompt for LLM.


## Generation

We will use a 4.1-nano to generate answers.

In [ ]:
from pstuts_rag.rag import RAGChainFactory

rag_factory = RAGChainFactory(retriever=retriever_factory.get_retriever())

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini",temperature=0)

In [ ]:
get_videos = rag_factory.get_rag_chain(llm)



In [ ]:
val = await get_videos.ainvoke({"question":"What are layers"})

In [ ]:
val.additional_kwargs

In [ ]:
val.pretty_print()

In [ ]:
val.content